In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")

In [2]:
df_read = pd.read_json("hf://datasets/HumanLLMs/Human-Like-DPO-Dataset/data.json")
data = pd.read_csv('generated_essays_merged.csv')

In [3]:
import sys
# 현재 주피터 커널이 사용하는 파이썬 환경에 pip 명령을 전달
!{sys.executable} -m pip install nltk
import re
import nltk
from collections import Counter

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab')


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: C:\Users\strow\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\strow\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [4]:
df_chosen = df_read[['chosen']]
df_rejected = df_read[['rejected']]
data_dpo = data[['HumanLLMs/Human-Like-LLama3-8B-Instruct_Generated_Essay']]
data_cft = data[['meta-llama/Meta-Llama-3-8B-Instruct_Generated_Essay']]
data_human = data[['Human_Written_Essay']]

def extract_features(text):
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text.lower())
    words_clean = [w for w in words if w.isalnum()]
    
    features = {}

    features['avg_sentence_length'] = len(words_clean) / max(len(sentences), 1)
    features['avg_word_length'] = np.mean([len(w) for w in words_clean]) if words_clean else 0
    
    # Complex words (>6 characters)
    complex_words = [w for w in words_clean if len(w) > 6]
    features['complex_word_ratio'] = len(complex_words) / max(len(words_clean), 1)
    
    return features

Wholedata = [df_chosen, df_rejected, data_dpo, data_cft, data_human]

updated_data = []

for i, df in enumerate(Wholedata):
    df.columns = ['text']
    
    # 기본 feature 추가
    df['text_length'] = df['text'].str.len()
    df['word_count'] = df['text'].str.split().str.len()
    
    # 이외의 feature 추출
    features_series = df['text'].apply(extract_features)
    features_df = pd.json_normalize(features_series)
    
    # 원본 df에 특징을 추가한 후, 업데이트된 리스트에 저장
    updated_df = pd.concat([df, features_df], axis=1)
    updated_data.append(updated_df)
    
# 반복문이 완료되면 updated_data 리스트에 모든 특징이 추가된 DataFrame들이 저장됩니다.
# 예: updated_data[0]이 df_chosen에 해당합니다.

print("--- 첫 번째 DataFrame (df_chosen)의 특징 추출 결과 확인 ---")
print(updated_data[0].head())


--- 첫 번째 DataFrame (df_chosen)의 특징 추출 결과 확인 ---
                                                text  text_length  word_count  \
0  😂 Ah, no I haven't! I'm dying to know, what's ...          125          26   
1  Oh, totally! 😄 I'm a sucker for a good ol' roc...          316          64   
2  😊 I'm actually a big fan of DIY projects! I'm ...          497          92   
3  Oh, man! I'm a total sucker for Italian food! ...          624         113   
4  You know, I've always been fascinated by music...          493          95   

   avg_sentence_length  avg_word_length  complex_word_ratio  
0             4.800000         3.541667            0.083333  
1             7.500000         3.700000            0.116667  
2             9.888889         4.134831            0.179775  
3             9.818182         4.259259            0.175926  
4            15.333333         3.978261            0.108696  


In [5]:
ALL_PRONOUNS = ['i', 'me', 'my', 'mine', 'we', 'us', 'our', 'ours', 
                'you', 'your', 'yours', 
                'he', 'him', 'his', 'she', 'her', 'hers', 'it', 'its', 
                'they', 'them', 'their', 'theirs', 
                'one', 'oneself', 'it\'s', 'he\'s', 'she\'s', 'they\'re'] 

def count_pronouns(text):
        # 대명사의 총 개수 세기
    words = re.findall(r'\b\w+\b', text.lower()) # 단어 단위로 분리
    
    pronoun_count = 0
    for word in words:
        if word in ALL_PRONOUNS:
            pronoun_count += 1
    return pronoun_count

for x in range(5):
    df = updated_data[x]  # 첫 번째 DataFrame을 선택
    
    # 1. 'pronoun_count' 열 추가
    #    apply 메서드를 사용하여 각 텍스트에 대한 대명사 개수를 계산
    df['pronoun_count'] = df['text'].apply(count_pronouns)
    
    # 2. 'pronoun_ratio' 열 계산
    #    총 대명사 수 / 총 단어 수 (word_count)
    #    총 단어 수(word_count)가 0인 경우를 방지하기 위해 1을 더해 나누거나, where를 사용합니다.
    df['pronoun_ratio'] = df['pronoun_count'] / df['word_count'].replace(0, 1)
    
    # 결과 확인 (선택 사항)
    print(df[['text', 'pronoun_ratio']].head())

                                                text  pronoun_ratio
0  😂 Ah, no I haven't! I'm dying to know, what's ...       0.115385
1  Oh, totally! 😄 I'm a sucker for a good ol' roc...       0.125000
2  😊 I'm actually a big fan of DIY projects! I'm ...       0.152174
3  Oh, man! I'm a total sucker for Italian food! ...       0.088496
4  You know, I've always been fascinated by music...       0.136842
                                                text  pronoun_ratio
0  I'm an artificial intelligence language model,...       0.115385
1  As a professional AI language model, I don't h...       0.106061
2  Good day. As a continuously evolving artificia...       0.104000
3  In accordance with my programming, I must emph...       0.077586
4  Good day. As a professional AI language model,...       0.060000
                                                text  pronoun_ratio
0  The bar chart reveals striking changes in the ...       0.003891
1  Rich countries often provide financial aid to

In [6]:
HEDGING_WORDS = [
    'might', 'could', 'may', 'can', # 모달 동사
    'possible', 'possibly', 'likely', 'unlikely', 'perhaps', 'maybe', # 가능성
    'suggests', 'seems', 'appears', 'indicates', 'arguable', 'arguably' # 주장 완화
]

def count_hedging_words(text):
    # 헤징 단어 개수 세기
    text_lower = text.lower()
    hedging_count = 0
    
    # 정확히 일치하는 단어만 카운트
    for word in HEDGING_WORDS:
        # 'i think'와 같은 구문도 포함하여 count
        if ' ' in word:
            hedging_count += text_lower.count(word)
        else:
            # 단어 경계를 사용하여 'might'가 'almighty' 같은 단어에 포함되지 않도록 함
            pattern = r'\b' + re.escape(word) + r'\b'
            hedging_count += len(re.findall(pattern, text_lower))
            
    return hedging_count

for x in range(5):
    df = updated_data[x]  # 첫 번째 DataFrame (df_chosen)을 선택
    
    # 헤징 단어 개수를 계산
    df['hedging_count'] = df['text'].apply(count_hedging_words)
    
    # 헤징 비율 계산
    df['hedging_ratio'] = df['hedging_count'] / df['word_count'].replace(0, 1)

In [7]:
nltk.download('averaged_perceptron_tagger_eng')

NOUN_TAGS = ['NN', 'NNS', 'NNP', 'NNPS'] 
VERB_TAGS = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']

def calculate_noun_verb_ratio(text):
    # 단어 동사 비율 계산
    words = nltk.word_tokenize(text)
    
    tagged_words = nltk.pos_tag(words)
    
    noun_count = 0
    verb_count = 0
    
    for word, tag in tagged_words:
        if tag in NOUN_TAGS:
            noun_count += 1
        elif tag in VERB_TAGS:
            verb_count += 1
            
    # Noun/Verb Ratio 계산
    if verb_count == 0:
        return noun_count 
        
    return noun_count / verb_count

for x in range(5):
    df = updated_data[x] 
    
    df['noun_verb_ratio'] = df['text'].apply(calculate_noun_verb_ratio)

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\strow\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [8]:
ADJECTIVE_TAGS = ['JJ', 'JJR', 'JJS']
NOUN_TAGS = ['NN', 'NNS', 'NNP', 'NNPS'] 

# 흔히 명사화를 나타내는 접미사 목록
NOMINALIZATION_SUFFIXES = [
    'tion', 'sion', 'ment', 'ness', 'ity', 
    'ism', 'ance', 'ence', 'ship', 'hood', 'age', 'al'
]

def calculate_nominalization_ratio(text):
    # 명사화 접미사를 가진 단어의 비율을 계산
    words_lower = nltk.word_tokenize(text.lower())
    nominal_count = 0
    
    for word in words_lower:
        # 단어가 정의된 접미사 중 하나로 끝나는지 확인
        if any(word.endswith(suffix) for suffix in NOMINALIZATION_SUFFIXES):
            nominal_count += 1
            
    total_words = len(words_lower)
    if total_words == 0:
        return 0
        
    return nominal_count / total_words


for x in range(5):
    df = updated_data[x] 

    df['nominalization_ratio'] = df['text'].apply(calculate_nominalization_ratio)

In [9]:
name = ['chosen', 'rejected', 'dpo', 'cft', 'human']

for x in range(5):
    updated_data[x].drop(columns = ['text', 'text_length', 'word_count', 'hedging_count', 'pronoun_count']).to_csv('data_'+name[x]+'.csv', index = False)